In [9]:
import numpy as np
import tensorflow as tf
import math
import random
from matplotlib import pyplot as plt

def sigmoid(x):
    return 1/(1+math.exp(-x))

def sigmoidderivative(x):
    return sigmoid(x) * (1-sigmoid(x))

def tanh(x):
    return np.tanh(x)

def relu(x):
    return max(0,x)

def tanh_derivative(x):
    return 1 - np.tanh(x)**2

def relu_derivative(x):
    if x>0:
        return 1
    else:
        return 0

vec_sigmoid = np.vectorize(sigmoid)
vec_sigmoidderivative = np.vectorize(sigmoidderivative)
vec_tanh = np.vectorize(tanh)
vec_tanhderivative = np.vectorize(tanh_derivative)
vec_relu = np.vectorize(relu)
vec_reluderivative = np.vectorize(relu_derivative)

In [10]:
def OneHot(n: np.uint8) -> np.ndarray:
    arr = np.zeros(10, dtype=np.uint8)
    arr[n] = 1
    return arr

In [11]:
class NeuralLayer:

    def __init__(self, numinputs:int, numoutputs:int, activation=None):

        self.numinputs = numinputs
        self.numoutputs = numoutputs
        self.activation = activation
        self.weights = np.random.randn(self.numoutputs, self.numinputs + 1)

    
    def Evaluate(self, inputs):

        inputs = np.append(inputs, np.array([1]))
    
        outputs = self.weights @ inputs # this is \vec{h}

        match self.activation:
            case "Sigmoid":
                outputs = vec_sigmoid(outputs)

            case "Softmax":
                denom = 0
                for i in range(len(outputs)):
                    denom += math.exp[outputs[i]]
                    outputs[i] = math.exp(outputs[i])
                outputs = outputs/denom

            case "ReLU":
                # Put code here, delete the pass
                #Alejandro shall not pass
                outputs = np.maximum(0, outputs)
            
            case "Tanh":
                # Put code here, delete the pass
                outputs = np.tanh(outputs)



        return outputs

    
        
    def ComputeLocalGradient(self, inputs):
        # z is output after activation
        # h is output after linear layer
        # w are weights
        # Need to compute three things:
        # dz/dh
        # dh/dw
        # dh/dx

        inputs = np.append(inputs, np.array([1]))
        outputs = self.weights @ inputs


        # This part computes dzdh, and has cases for various activation functions
        match self.activation:
            case "Sigmoid":
                dzdh = np.diag(vec_sigmoidderivative(outputs))
            case "Softmax":
                n = len(outputs)
                dzdh = np.zeros((n, n))
                denom = 0
                for i in range(n):
                    denom += math.exp(outputs[i])
                
                for i in range(n):
                    for j in range(n):
                        if i == j:
                            dzdh[i][j] = (denom * math.exp(outputs[i]) - (math.exp(outputs[i])**2))/(denom**2)
                        else:
                            dzdh[i][j] = -(math.exp(outputs[j]))*(math.exp(outputs[j]))/(denom**2)

            case "ReLU":
                # Put code here and remove pass
                dzdh = np.diag(np.where(outputs > 0, 1, 0))
            case "Tanh":
                # Put code here and remove pass
                dzdh = np.diag(1 - np.tanh(outputs**2))



            

        
        
        
        
        # This part computes dhdw        
        dhdw = np.zeros((self.numoutputs, self.numoutputs, self.numinputs+1)) #because of bias
        for i in range(self.numoutputs):
            for j in range(self.numinputs):
                dhdw[i,i,j] = inputs[i]
            dhdw[i,i,self.numinputs] = 1
            
        # This part computes dhdx
        dhdx = self.weights[:, :-1]


        return (dzdh, dhdw, dhdx)

    
    

        

In [12]:
Layer1 = NeuralLayer(5,3,"Sigmoid")
#Layer1.weights = np.array([[1, 2, -1], [3, -2, 1]])
test1 = np.array([1, 2,3,4,5])
(dzdh, dhdw, dhdx) = Layer1.ComputeLocalGradient(test1)
print(dzdh.shape)
print(dhdw.shape)
print(dhdx.shape)

(3, 3)
(3, 3, 6)
(3, 5)


In [13]:
class NeuralNetwork:

    def __init__(self, errorfunc=None):
        
        self.errorfunc = errorfunc
        self.layers = []
        self.numlayers = 0

    def AppendLayer(self, layer: NeuralLayer):
        # need to check that the new layer to be appended has same 
        # number of inputs as the last layer already in the network
        if len(self.layers) > 0:
            if layer.numinputs == self.layers[-1].numoutputs:
                self.layers.append(layer)
                self.numlayers += 1
            else:
                print("Error: number of inputs does not match previous layer")
        else:
            self.layers.append(layer)
            self.numlayers += 1

        
    def Evaluate(self, inputs):

        outputs = []
        outputs.append(self.layers[0].Evaluate(inputs))

        for i in range(1,self.numlayers):
            outputs.append(self.layers[i].Evaluate(outputs[i-1]))
        
        return outputs

    def ComputeError(self, inputs, trueoutputs):

        outputs = self.Evaluate(inputs)
        
        if self.errorfunc == "MSE":
            n = len(outputs[-1])
            diffs = outputs[-1] - trueoutputs
            err = np.dot(diffs, diffs)
            err = err/(2*n)
            return err

    def BackPropagate(self, inputs, trueoutputs, learningrate):

        outputs = self.Evaluate(inputs)
        gradients = []

        # Compute all the necessary gradients
        for i in range(self.numlayers):
            if i == 0:
                tempinput = inputs
            else:
                tempinput = outputs[i-1]
            
            gradients.append(self.layers[i].ComputeLocalGradient(tempinput))

        match self.errorfunc:
            case "MSE":
                dldz = (0.5) * (outputs[-1] - trueoutputs)

            case "CrossEntropy":
                dldz = np.zeros(len(trueoutputs))
                spot = np.where(1 == trueoutputs)
                dldz[spot] = 1/outputs[-1][spot]
                
            

        # Update weights, working backwards

        currgrad = dldz @ gradients[-1][0]
    
        for i in range(self.numlayers-1, -1, -1):
            self.layers[i].weights -= learningrate * (currgrad @ gradients[i][1])
            currgrad = currgrad @ gradients[i][0] @ gradients[i][2]

In [15]:
MyNN = NeuralNetwork(errorfunc="MSE")
MyLayer1 = NeuralLayer(5, 3, "Sigmoid")
MyLayer2 = NeuralLayer(3, 2, "Sigmoid")


MyNN.AppendLayer(MyLayer1)
MyNN.AppendLayer(MyLayer2)

myinput = np.array([1,2,3,4,5])
mytrue = np.array([1,0])

print(MyNN.ComputeError(myinput, mytrue))

for i in range(10):
    MyNN.BackPropagate(myinput, mytrue, 1)
    print(MyNN.ComputeError(myinput, mytrue))




0.017855449612622438
0.01664821266278719
0.015682851637275928
0.014889868303736915
0.014223862366432296
0.013654014058041754
0.013158709817746451
0.012722366655767883
0.012333479296715041
0.011983376570671994
0.011665404838636413


In [20]:
# Load data from MNIST database
(x_train0, y_train0), (x_test0, y_test0) = tf.keras.datasets.mnist.load_data()
assert x_train0.shape == (60000, 28, 28)
assert x_test0.shape == (10000, 28, 28)
assert y_train0.shape == (60000,)
assert y_test0.shape == (10000,)

# Prepare data for processing
# x_train and x_test need to be reshaped and converted to np.float64
# y_train and y_test need to be one-hot encoded

x_train = []
y_train = []

for y in range(6000):
    images = x_train0[y].reshape(28*28).astype(np.float64)
    images = images/255.0
    x_train.append(images)

    y_train.append(OneHot(y_train0[y]))


x_test = []
y_test = []

for x in range(1000):
    images = x_test0[x].reshape(28*28).astype(np.float64)
    images = j/255.0
    x_test.append(images)

    y_test.append(OneHot(y_test0[x]))

print(f"y_train length: {len(y_train)}, shape of first element: {y_train[0].shape}")
print(f"x_train length: {len(x_train)}, shape of first element: {x_train[0].shape}")
print(f"x_test length: {len(x_test)}, shape of first element: {x_test[0].shape}")
print(f"y_test length: {len(y_test)}, shape of first element: {y_test[0].shape}")
print(f"\ne.g. label: {y_train0[0]} -> Onehot: {y_train[0]}")

y_train length: 6000, shape of first element: (10,)
x_train length: 6000, shape of first element: (784,)
x_test length: 1000, shape of first element: (784,)
y_test length: 1000, shape of first element: (10,)

e.g. label: 5 -> Onehot: [0 0 0 0 0 1 0 0 0 0]


In [18]:
x_train0[0].reshape(28*28)


array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [19]:
MyMNISTNetwork = NeuralNetwork("MSE")
MyMNISTNetwork.AppendLayer(NeuralLayer(28*28,10,"Sigmoid"))


y_train0[0]

#testinput = np.astype(x_train0[0].reshape(28*28), np.float64)
testinput = x_train0[0].reshape(28*28)
testinput = testinput.astype(np.float64)

testinput /= 255.0
#print(testinput.sum())
#print(MyMNISTNetwork.Evaluate(testinput))
#print(MyMNISTNetwork.layers[-1].weights.dtype)

onehot = np.array([0,0,0,0,0,1,0,0,0,0])

print(MyMNISTNetwork.ComputeError(testinput, onehot))
for i in range(100):
    MyMNISTNetwork.BackPropagate(testinput, onehot, 10)
    print(MyMNISTNetwork.ComputeError(testinput, onehot))
    
print(MyMNISTNetwork.ComputeError(testinput, onehot))

print("Final check evaluation: " + str(MyMNISTNetwork.Evaluate(testinput)))

0.12926952831670788
0.11920614972365635
0.10974446462647724
0.10506988926572218
0.10320285080023736
0.10231082595498198
0.1018052367530684
0.10148367319967784
0.10126233784610342
0.101101090804367
0.10097851476027368
0.10088221299121865
0.10080454180255786
0.10074054461094754
0.10068687258854643
0.10064118471247582
0.10060179675610281
0.10056746689003619
0.10053725983088709
0.10051045799390532
0.10048650176272197
0.10046494834936104
0.10044544284390486
0.10042769744827844
0.10041147632232779
0.10039658435249928
0.10038285870974355
0.1003701624220108
0.10035837942277122
0.10034741069521969
0.1003371712396858
0.10032758766644675
0.10031859626858541
0.10031014146686176
0.10030217454545545
0.10029465261703521
0.10028753777004248
0.10028079636181837
0.10027439842926182
0.10026831719481985
0.10026252865027259
0.10025701120436992
0.10025174538316
0.10024671357402684
0.10024189980616079
0.10023728956154085
0.10023286961158188
0.1002286278754612
0.10022455329683153
0.10022063573618809
0.1002168